<img width="8%" alt="Twitter.jpeg" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Twitter.jpeg" style="border-radius: 15%">

# Twitter - Get tweets by username
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Twitter+-+Get+tweets+by+username:+Error+short+description">Bug report</a>

**Tags:** #twitter #username #snippet #content #dataframe

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-07-22 (Created: 2024-07-17)

**Description:** This notebook allows users to get all tweets made by inputed twitter username.

### How to retrive API key with apify

1. Go to https://apify.com.
2. Click "Sign up for free" and use your google account to sign up.
3. Once your account has been created, navigate to "Settings" on the left panel of the screen.
4. Here you will click on the tab labeled "Integrations" where your personal API token that was automatically generated with sign up will be.
5. Copy that token and use it to extract data!

## Input

### Import libraries

In [1]:
import requests
import time
import pandas as pd

### Setup variables

- `APIFY_API_TOKEN`: personal token apify creates to access data
- `TWITTER_URL`: link to the twitter account
- `OUTPUT_CSV`: excel file

In [2]:
APIFY_API_TOKEN = 'apify_api_gXWnLEPiE7wC8ALUwQkJ0QcdbuQzU84xxxxx'
TWITTER_URL = 'https://twitter.com/Spotify'
OUTPUT_CSV = f"{TWITTER_URL.split('https://twitter.com/')[1].replace('/', '_')}_tweets.csv"

## Model

### Setup apify actor

In [3]:
# Prepare the actor input
actor_input = {
    "startUrls": [{"url": TWITTER_URL}],
    "resultsLimit": 100,  # Adjust the limit as needed
    "mode": "own"  # Options: "own", "replies", "both"
}

# Run the actor
run_actor_url = f'https://api.apify.com/v2/acts/quacker~twitter-scraper/runs?token={APIFY_API_TOKEN}'
run_response = requests.post(run_actor_url, json=actor_input)
run_response.raise_for_status()  # Raise an error if the request failed
run = run_response.json()
run_id = run['data']['id']

### Get tweets data

In [4]:
while True:
    run_details_url = f'https://api.apify.com/v2/actor-runs/{run_id}?token={APIFY_API_TOKEN}'
    run_details_response = requests.get(run_details_url)
    run_details_response.raise_for_status()
    run_details = run_details_response.json()
    if run_details['data']['status'] in ['SUCCEEDED', 'FAILED', 'ABORTED']:
        break
    time.sleep(10)  # Wait for 10 seconds before checking the status again

# Fetch the results
dataset_id = run_details['data']['defaultDatasetId']
dataset_items_url = f'https://api.apify.com/v2/datasets/{dataset_id}/items?token={APIFY_API_TOKEN}'
dataset_items_response = requests.get(dataset_items_url)
dataset_items_response.raise_for_status()
dataset_items = dataset_items_response.json()

### Dataframe structure function

In [6]:
def get_tweets(
    tid,
    text,
    reply_count,
    retweet_count,
    favorites_count,
    hashtags,
    user_mentions,
    url,
    created_at,
    view_count
):
    return {
        "ID": tid,
        "TEXT": text,
        "REPLY_COUNT": reply_count,
        "RETWEET_COUNT": retweet_count,
        "FAVORITES_COUNT": favorites_count,
        "HASHTAGS": hashtags,
        "USER_MENTIONS": user_mentions,
        "URL": url,
        "CREATED_AT": created_at,
        "VIEW_COUNT": view_count
    }

## Output

### Display

In [10]:
data = []

for tweet in dataset_items:
    data_tweet = get_tweets(
            tweet["id"],
            tweet["full_text"],
            tweet["reply_count"],
            tweet["retweet_count"],
            tweet["favorite_count"],
            tweet["hashtags"],
            tweet["user_mentions"],
            tweet["url"],
            tweet["created_at"],
            tweet["view_count"]
        )
    data.append(data_tweet)
        
df = pd.DataFrame(data)
df

### Save to csv file

In [11]:
df.to_csv(OUTPUT_CSV, index=False)